In [1]:
pip install selenium beautifulsoup4 pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# Path to ChromeDriver
path = "C:\\Users\\Khoirunisa Jannatuzzahra\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# URL to be accessed
url_lowest_rated = "https://www.imdb.com/chart/bottom/?ref_=chttentp_ql_7"

# IF YOU WANT TO DO HEADLESS BROWSER, use codes below
options = Options()
options.headless = False  # Change to True if you want to run the browser in headless mode
service = Service(path)

try:
    # THIS IS FOR NORMAL BROWSER SESSION (NOT HEADLESS)
    # Define browser to get the first URL
    browser = webdriver.Chrome(service=service, options=options)

    # Define browser to get the second/detail URL
    browser2 = webdriver.Chrome(service=service, options=options)

    # Check file availability and content is not empty
    filename = "imdb_1.csv"
    filename2 = "imdb_2.csv"

    def visualize_data():
        print("Visualize Something")

    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        os.remove(filename)
        os.remove(filename2)

    # Initialize file handlers outside try block
    f = open(filename, "w")
    fheaders = "Name,Year,Durasi(Menit),Rating\n"
    f2 = open(filename2, "w")
    f2headers = "Name,Budget,Gross_US,Opening_Week,Open_Week_Date,Gross_World,Runtime,Color,Sound_Mix,Aspect_Ratio\n"

    f.write(fheaders)
    f2.write(f2headers)

    try:
        # ACTION 1
        # Open session for first URL
        browser.get(url_lowest_rated)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-b189961a-0")))

        first_page = BeautifulSoup(browser.page_source, 'html.parser')
        movies = first_page.find_all("div", {"class": "sc-b189961a-0 hBZnfJ cli-children"})

        count2 = 0
        print("PRIMARY DATA")
        print("-----------------------------------")
        for movie in movies:
            count2 += 1
            menit = 0
            rating = "Not Rated"
            name = re.sub(r"\d+. ", "", movie.find("h3", {"class": "ipc-title__text"}).text)
            detail_line = movie.findAll("span", {"class": "sc-b189961a-8 kLaxqf cli-title-metadata-item"})
            if len(detail_line) > 2:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
                rating = detail_line[2].text
            elif len(detail_line) > 1:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
            else:
                year_movie = detail_line[0].text

            print("-----------------------------------")
            print(name + "," + year_movie + "," + str(menit) + "," + rating)
            f.write(name + "," + year_movie + "," + str(menit) + "," + rating + "\n")

        # ACTION 2
        links = browser.find_elements(By.XPATH, '//div[@class="sc-b189961a-0 hBZnfJ cli-children"]')

        count = 0
        print("-----------------------------------")
        print("-----------------------------------")
        print("SECONDARY DATA")
        print("panjang ", len(links))
        movie_titles = first_page.find_all("div", {"class": "ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN cli-title"})

        for link in links:
            title = re.sub(r"\d+. ", "", movie_titles[count].text)
            print("judul ke-" + str(count + 1) + ": " + title)
            count += 1
            budget_num = 0
            gross_us_num = 0
            open_week_rev_num = 0
            open_week_date_std = "1900-01-01 00:00:00"
            gross_world_num = 0

            print("----------------------")
            print(count, link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'), "\n")
            browser2.get(link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'))

            # BOX OFFICE DATA ON THE 2ND URL
            det_page = browser2.page_source
            container_rows = BeautifulSoup(det_page, "html.parser")
            box_office_elements = container_rows.find("div", {"data-testid": "title-boxoffice-section"})
            tech_specs_elements = container_rows.find("div", {"data-testid": "title-techspecs-section"})
            
            if box_office_elements is not None:
                det_movie = box_office_elements.find_all("span", {"class": "ipc-metadata-list-item__list-content-item"})

                # Initialize variables with default values
                budget = "NA"
                gross_us = "NA"
                open_week_rev = "NA"
                open_week_date = "NA"
                gross_world = "NA"

                if len(det_movie) > 0:
                    budget = det_movie[0].text.strip() if det_movie[0].text.strip() else "NA"
                if len(det_movie) > 1:
                    gross_us = det_movie[1].text.strip() if det_movie[1].text.strip() else "NA"
                if len(det_movie) > 2:
                    open_week_rev = det_movie[2].text.strip() if det_movie[2].text.strip() else "NA"
                if len(det_movie) > 3:
                    open_week_date = det_movie[3].text.strip() if det_movie[3].text.strip() else "NA"
                if len(det_movie) > 4:
                    gross_world = det_movie[4].text.strip() if det_movie[4].text.strip() else "NA"

                print("budget = " + budget)
                print("gross_us = " + gross_us)
                print("opening_week_rev = " + open_week_rev)
                print("opening_week_date = " + open_week_date)
                print("gross_world = " + gross_world)

                print("=====================================")
                try:
                    budget_num = int(re.sub("[A-Z£€₹$,()a-z]", "", budget)) if budget != "NA" else 0
                except ValueError:
                    budget_num = 0
                try:
                    gross_us_num = int(re.sub("[A-Z£€₹$,()a-z]", "", gross_us)) if gross_us != "NA" else 0
                except ValueError:
                    gross_us_num = 0
                try:
                    open_week_rev_num = int(re.sub("[A-Z£€₹$,()a-z]", "", open_week_rev)) if open_week_rev != "NA" else 0
                except ValueError:
                    open_week_rev_num = 0
                try:
                    open_week_date_std = datetime.strptime(open_week_date, "%b %d, %Y") if open_week_date != "NA" else datetime.strptime("1900-01-01", "%Y-%m-%d")
                except ValueError:
                    open_week_date_std = datetime.strptime("1900-01-01", "%Y-%m-%d")
                try:
                    gross_world_num = int(re.sub("[A-Z£€₹$,()a-z]", "", gross_world)) if gross_world != "NA" else 0
                except ValueError:
                    gross_world_num = 0

                print("Data has been formatted to standard")
                print("-----------------------------------")
                print("budget =", budget_num)
                print("gross_us =", gross_us_num)
                print("opening_week_rev =", open_week_rev_num)
                print("opening_week_date =", open_week_date_std)
                print("gross_world =", gross_world_num)

                # TECHNICAL SPECS DATA
                runtime = "NA"
                color = "NA"
                sound_mix = "NA"
                aspect_ratio = "NA"
                
                if tech_specs_elements is not None:
                    runtime_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_runtime"})
                    color_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_color"})
                    sound_mix_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_soundmix"})
                    aspect_ratio_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_aspectratio"})
        
                    if runtime_element:
                        runtime_text = runtime_element.find("div", {"class": "ipc-metadata-list-item__content-container"})
                        runtime_text = runtime_text.text.strip() if runtime_text else "NA"
                        # Konversi runtime ke menit
                        runtime = 0
                        if 'h' in runtime_text and 'min' in runtime_text:
                            hours, minutes = map(int, re.findall(r'\d+', runtime_text))
                            runtime = hours * 60 + minutes
                        elif 'h' in runtime_text:
                            runtime = int(re.findall(r'\d+', runtime_text)[0]) * 60
                        elif 'min' in runtime_text:
                            runtime = int(re.findall(r'\d+', runtime_text)[0])
                    
                    color = color_element.find("div", {"class": "ipc-metadata-list-item__content-container"}).text.strip() if color_element else "NA"
                    
                    sound_mix_elements = sound_mix_element.find_all("li", class_="ipc-inline-list__item") if sound_mix_element else []
                    if sound_mix_elements:
                        sound_mix = "; ".join([item.text.strip() for item in sound_mix_elements])
                    else:
                        sound_mix = "NA"
                    
                    aspect_ratio_elements = aspect_ratio_element.find_all("li", class_="ipc-inline-list__item") if aspect_ratio_element else []
                    if aspect_ratio_elements:
                        aspect_ratio = "; ".join([item.text.strip() for item in aspect_ratio_elements])
                    else:
                        aspect_ratio = "NA"
                    
                    print("runtime = " + str(runtime))
                    print("color = " + color)
                    print("sound_mix = " + sound_mix)
                    print("aspect_ratio = " + aspect_ratio)

                    # Write the data even if some fields are missing
                    f2.write(f"{title},{budget_num},{gross_us_num},{open_week_rev_num},{open_week_date_std},{gross_world_num},{runtime},{color},{sound_mix},{aspect_ratio}\n")

    except Exception as E:
        print(E)
    finally:
        f.close()
        f2.close()
        browser.quit()
        browser2.quit()
except Exception as ex:
    print("Failed to start browser or driver:", ex)

PRIMARY DATA
-----------------------------------
-----------------------------------
Disaster Movie,2008,87,PG-13
-----------------------------------
Manos: The Hands of Fate,1966,70,Not Rated
-----------------------------------
Birdemic: Shock and Terror,2010,105,Not Rated
-----------------------------------
Superbabies: Baby Geniuses 2,2004,88,PG
-----------------------------------
Kirk Cameron's Saving Christmas,2014,79,PG
-----------------------------------
The Hottie & the Nottie,2008,91,PG-13
-----------------------------------
House of the Dead,2003,90,R
-----------------------------------
Son of the Mask,2005,94,PG
-----------------------------------
Epic Movie,2007,86,PG-13
-----------------------------------
Radhe,2021,109,TV-MA
-----------------------------------
Pledge This!,2006,91,R
-----------------------------------
Battlefield Earth,2000,117,PG-13
-----------------------------------
Alone in the Dark,2005,96,R
-----------------------------------
Dragonball Evolution,20

In [3]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# Path to ChromeDriver
path = "C:\\Users\\Khoirunisa Jannatuzzahra\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# URL to be accessed
url_lowest_rated = "https://www.imdb.com/chart/bottom/?ref_=chttentp_ql_7"

# IF YOU WANT TO DO HEADLESS BROWSER, use codes below
options = Options()
options.headless = False  # Change to True if you want to run the browser in headless mode
service = Service(path)

try:
    # THIS IS FOR NORMAL BROWSER SESSION (NOT HEADLESS)
    # Define browser to get the first URL
    browser = webdriver.Chrome(service=service, options=options)

    # Define browser to get the second/detail URL
    browser2 = webdriver.Chrome(service=service, options=options)

    # Check file availability and content is not empty
    filename = "imdbdata_1.csv"
    filename2 = "imdbdata_2.csv"

    def visualize_data():
        print("Visualize Something")

    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        os.remove(filename)
        os.remove(filename2)

    # Initialize file handlers outside try block
    f = open(filename, "w")
    fheaders = "Name,Year,Durasi(Menit),Rating\n"
    f2 = open(filename2, "w")
    f2headers = "Name,Budget,Gross_US,Opening_Week,Open_Week_Date,Gross_World,Runtime,Color,Sound_Mix,Aspect_Ratio\n"

    f.write(fheaders)
    f2.write(f2headers)

    try:
        # ACTION 1
        # Open session for first URL
        browser.get(url_lowest_rated)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-b189961a-0")))

        first_page = BeautifulSoup(browser.page_source, 'html.parser')
        movies = first_page.find_all("div", {"class": "sc-b189961a-0 hBZnfJ cli-children"})

        count2 = 0
        print("PRIMARY DATA")
        print("-----------------------------------")
        for movie in movies:
            count2 += 1
            menit = 0
            rating = "Not Rated"
            name = re.sub(r"\d+. ", "", movie.find("h3", {"class": "ipc-title__text"}).text)
            detail_line = movie.findAll("span", {"class": "sc-b189961a-8 kLaxqf cli-title-metadata-item"})
            if len(detail_line) > 2:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
                rating = detail_line[2].text
            elif len(detail_line) > 1:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
            else:
                year_movie = detail_line[0].text

            print("-----------------------------------")
            print(name + "," + year_movie + "," + str(menit) + "," + rating)
            f.write(name + "," + year_movie + "," + str(menit) + "," + rating + "\n")

        # ACTION 2
        links = browser.find_elements(By.XPATH, '//div[@class="sc-b189961a-0 hBZnfJ cli-children"]')

        count = 0
        print("-----------------------------------")
        print("-----------------------------------")
        print("SECONDARY DATA")
        print("panjang ", len(links))
        movie_titles = first_page.find_all("div", {"class": "ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN cli-title"})

        for link in links:
            title = re.sub(r"\d+. ", "", movie_titles[count].text)
            print("judul ke-" + str(count + 1) + ": " + title)
            count += 1
            budget_num = 0
            gross_us_num = 0
            open_week_rev_num = 0
            open_week_date_std = "1900-01-01 00:00:00"
            gross_world_num = 0

            print("----------------------")
            print(count, link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'), "\n")
            browser2.get(link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'))

            # BOX OFFICE DATA ON THE 2ND URL
            det_page = browser2.page_source
            container_rows = BeautifulSoup(det_page, "html.parser")
            box_office_elements = container_rows.find("div", {"data-testid": "title-boxoffice-section"})
            tech_specs_elements = container_rows.find("div", {"data-testid": "title-techspecs-section"})
            
            if box_office_elements is not None:
                det_movie = box_office_elements.find_all("span", {"class": "ipc-metadata-list-item__list-content-item"})

                # Initialize variables with default values
                budget = "NA"
                gross_us = "NA"
                open_week_rev = "NA"
                open_week_date = "NA"
                gross_world = "NA"

                if len(det_movie) > 0:
                    budget = det_movie[0].text.strip() if det_movie[0].text.strip() else "NA"
                if len(det_movie) > 1:
                    gross_us = det_movie[1].text.strip() if det_movie[1].text.strip() else "NA"
                if len(det_movie) > 2:
                    open_week_rev = det_movie[2].text.strip() if det_movie[2].text.strip() else "NA"
                if len(det_movie) > 3:
                    open_week_date = det_movie[3].text.strip() if det_movie[3].text.strip() else "NA"
                if len(det_movie) > 4:
                    gross_world = det_movie[4].text.strip() if det_movie[4].text.strip() else "NA"

                print("budget = " + budget)
                print("gross_us = " + gross_us)
                print("opening_week_rev = " + open_week_rev)
                print("opening_week_date = " + open_week_date)
                print("gross_world = " + gross_world)

                print("=====================================")
                try:
                    budget_num = int(re.sub("[A-Z£€₹$,()a-z]", "", budget)) if budget != "NA" else 0
                except ValueError:
                    budget_num = 0
                try:
                    gross_us_num = int(re.sub("[A-Z£€₹$,()a-z]", "", gross_us)) if gross_us != "NA" else 0
                except ValueError:
                    gross_us_num = 0
                try:
                    open_week_rev_num = int(re.sub("[A-Z£€₹$,()a-z]", "", open_week_rev)) if open_week_rev != "NA" else 0
                except ValueError:
                    open_week_rev_num = 0
                try:
                    open_week_date_std = datetime.strptime(open_week_date, "%b %d, %Y") if open_week_date != "NA" else datetime.strptime("1900-01-01", "%Y-%m-%d")
                except ValueError:
                    open_week_date_std = datetime.strptime("1900-01-01", "%Y-%m-%d")
                try:
                    gross_world_num = int(re.sub("[A-Z£€₹$,()a-z]", "", gross_world)) if gross_world != "NA" else 0
                except ValueError:
                    gross_world_num = 0

                print("Data has been formatted to standard")
                print("-----------------------------------")
                print("budget =", budget_num)
                print("gross_us =", gross_us_num)
                print("opening_week_rev =", open_week_rev_num)
                print("opening_week_date =", open_week_date_std)
                print("gross_world =", gross_world_num)

            # TECHNICAL SPECS DATA
            runtime = "NA"
            color = "NA"
            sound_mix = "NA"
            aspect_ratio = "NA"
            
            if tech_specs_elements is not None:
                runtime_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_runtime"})
                color_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_color"})
                sound_mix_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_soundmix"})
                aspect_ratio_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_aspectratio"})
    
                if runtime_element:
                    runtime_text = runtime_element.find("div", {"class": "ipc-metadata-list-item__content-container"})
                    runtime_text = runtime_text.text.strip() if runtime_text else "NA"
                    # Konversi runtime ke menit
                    runtime = 0
                    if 'h' in runtime_text and 'min' in runtime_text:
                        hours, minutes = map(int, re.findall(r'\d+', runtime_text))
                        runtime = hours * 60 + minutes
                    elif 'h' in runtime_text:
                        runtime = int(re.findall(r'\d+', runtime_text)[0]) * 60
                    elif 'min' in runtime_text:
                        runtime = int(re.findall(r'\d+', runtime_text)[0])
                
                color = color_element.find("div", {"class": "ipc-metadata-list-item__content-container"}).text.strip() if color_element else "NA"
                
                sound_mix_elements = sound_mix_element.find_all("li", class_="ipc-inline-list__item") if sound_mix_element else []
                if sound_mix_elements:
                    sound_mix = "; ".join([item.text.strip() for item in sound_mix_elements])
                else:
                    sound_mix = "NA"
                
                aspect_ratio_elements = aspect_ratio_element.find_all("li", class_="ipc-inline-list__item") if aspect_ratio_element else []
                if aspect_ratio_elements:
                    aspect_ratio = "; ".join([item.text.strip() for item in aspect_ratio_elements])
                else:
                    aspect_ratio = "NA"
                
                print("runtime = " + str(runtime))
                print("color = " + color)
                print("sound_mix = " + sound_mix)
                print("aspect_ratio = " + aspect_ratio)

            # Write the data even if some fields are missing
            f2.write(f"{title},{budget_num},{gross_us_num},{open_week_rev_num},{open_week_date_std},{gross_world_num},{runtime},{color},{sound_mix},{aspect_ratio}\n")

    except Exception as E:
        print(E)
    finally:
        f.close()
        f2.close()
        browser.quit()
        browser2.quit()
except Exception as ex:
    print("Failed to start browser or driver:", ex)

PRIMARY DATA
-----------------------------------
-----------------------------------
Disaster Movie,2008,87,PG-13
-----------------------------------
Manos: The Hands of Fate,1966,70,Not Rated
-----------------------------------
Birdemic: Shock and Terror,2010,105,Not Rated
-----------------------------------
Superbabies: Baby Geniuses 2,2004,88,PG
-----------------------------------
Kirk Cameron's Saving Christmas,2014,79,PG
-----------------------------------
The Hottie & the Nottie,2008,91,PG-13
-----------------------------------
House of the Dead,2003,90,R
-----------------------------------
Son of the Mask,2005,94,PG
-----------------------------------
Epic Movie,2007,86,PG-13
-----------------------------------
Radhe,2021,109,TV-MA
-----------------------------------
Pledge This!,2006,91,R
-----------------------------------
Battlefield Earth,2000,117,PG-13
-----------------------------------
Alone in the Dark,2005,96,R
-----------------------------------
Dragonball Evolution,20

In [4]:
# MERGE IMDB
import pandas as pd
fn1 = 'imdbdata_1.csv'
fn2 = 'imdbdata_2.csv'

fn3 = 'imdbdata_3.csv'

if os.path.isfile(fn3) and os.path.getsize(fn3) > 0:      
    os.remove(fn3)
    
# Read CSV files into dataframes
df1 = pd.read_csv(fn1)
df2 = pd.read_csv(fn2)

merged = df1.merge(df2,on='Name')
merged.to_csv("imdbdata_3.csv", index=False)